In [127]:
import pandas as pd

from bs4 import BeautifulSoup

In [180]:
dataset = pd.read_csv(r'C:\Users\JH\Desktop\Final\CBA\Data\kind_bankrupt_Final.csv', index_col=0)
train = pd.read_csv(r'C:\Users\JH\Desktop\Final\CBA\Data\dataset.csv', index_col = 0)
print(train.shape, dataset.shape)

(27357, 271) (938, 6)


In [181]:
textbox = []
for i in range(len(dataset)):
    html = dataset['text'][i]
    soup = BeautifulSoup(html, 'html.parser').get_text()
    
    if len(html[soup.find('5.'):soup.find('6.')]) > 1:
        textbox.append(html[soup.find('5.'):soup.find('6.')])
    else:
        textbox.append(html[soup.find('2.'): soup.find('3.')])


In [182]:
textbox2 = []
for i in range(len(textbox)):
    a = textbox[i].split('\n')
    try:
        textbox2.append(a[1])
    except:
        textbox2.append('')

In [183]:
dataset['폐지사유'] = textbox
dataset['폐지사유요약'] = textbox2
dataset

,회사명,거래소코드,공시발생일,공시제목,text,link,폐지사유,폐지사유요약
0,주은리스,030970,2002-11-28 18:54,상장폐지,\n\n\n\n\n .document\t{FONT-FAMILY:바탕; FONT-S...,https://kind.krx.co.kr/external/2002/11/28/K00...,5.등록취소사유\n액면가액일정비율 미달로 연속하여 10일간 지속\n\n\n,액면가액일정비율 미달로 연속하여 10일간 지속
1,글로벌파워,064910,2002-11-26 18:53,상장폐지,\n\n\n\n\n .document\t{FONT-FAMILY:바탕; FONT-S...,https://kind.krx.co.kr/external/2002/11/26/K00...,5.등록취소사유\n증권투자회사법 제64조의 규정에 의한 해산사유의 발생\n\n\n,증권투자회사법 제64조의 규정에 의한 해산사유의 발생
2,주은리스,030970,2002-11-21 18:24,상장폐지승인,\n\n\n\n\n .document\t{FONT-FAMILY:바탕; FONT-S...,https://kind.krx.co.kr/external/2002/11/21/K00...,5.등록취소사유\n액면가액일정비율 미달로 연속하여 10일간 지속\n\n\n,액면가액일정비율 미달로 연속하여 10일간 지속
3,정산애강,022220,2002-11-13 16:59,상장폐지,\n\n\n\n\n .document\t{FONT-FAMILY:바탕; FONT-S...,https://kind.krx.co.kr/external/2002/11/13/K00...,5.등록취소사유\n최종부도 및 은행거래정지\n\n\n,최종부도 및 은행거래정지
4,소프트윈,049790,2002-11-06 18:09,상장폐지((주)소프트윈),\n\n\n\n\n .document\t{FONT-FAMILY:바탕; FONT-S...,https://kind.krx.co.kr/external/2002/11/06/K00...,5.등록취소사유\n최종부도 및 은행거래정지\n\n\n,최종부도 및 은행거래정지
...,...,...,...,...,...,...,...,...
933,유안타제2호스팩,219960,2018-02-08 17:51,상장폐지,\n\n\n\n\n\n\r\n\t.PGBRK\t\t\t{page-break-afte...,https://kind.krx.co.kr/external/2018/02/08/001...,5.상장폐지사유\n상장예비심사 청구서 미제출로 관리종목 지정 후 1개월 이내 동 사...,상장예비심사 청구서 미제출로 관리종목 지정 후 1개월 이내 동 사유 미해소
934,셀트리온,068270,2018-02-07 15:48,상장폐지(유가증권시장 상장),\n\n\n\n\n\n\r\n\t.PGBRK\t\t\t{page-break-afte...,https://kind.krx.co.kr/external/2018/02/07/000...,5.상장폐지사유\n유가증권시장 상장\n\n\n,유가증권시장 상장
935,키움스팩3호,218710,2018-02-01 18:08,상장폐지,\n\n\n\n\n\n\r\n\t.PGBRK\t\t\t{page-break-afte...,https://kind.krx.co.kr/external/2018/02/01/000...,5.상장폐지사유\n상장예비심사 청구서 미제출로 관리종목 지정 후 1개월 이내 동 사...,상장예비심사 청구서 미제출로 관리종목 지정 후 1개월 이내 동 사유 미해소
936,골든브릿지제3호스팩,219580,2018-01-25 18:10,상장폐지,\n\n\n\n\n\n\r\n\t.PGBRK\t\t\t{page-break-afte...,https://kind.krx.co.kr/external/2018/01/25/000...,5.상장폐지사유\n상장예비심사 청구서 미제출로 관리종목 지정 후 1개월 이내 동 사...,상장예비심사 청구서 미제출로 관리종목 지정 후 1개월 이내 동 사유 미해소


In [184]:
dataset['key'] = dataset['거래소코드'] + dataset.공시발생일.str[0:4]
dataset

,회사명,거래소코드,공시발생일,공시제목,text,link,폐지사유,폐지사유요약,key
0,주은리스,030970,2002-11-28 18:54,상장폐지,\n\n\n\n\n .document\t{FONT-FAMILY:바탕; FONT-S...,https://kind.krx.co.kr/external/2002/11/28/K00...,5.등록취소사유\n액면가액일정비율 미달로 연속하여 10일간 지속\n\n\n,액면가액일정비율 미달로 연속하여 10일간 지속,0309702002
1,글로벌파워,064910,2002-11-26 18:53,상장폐지,\n\n\n\n\n .document\t{FONT-FAMILY:바탕; FONT-S...,https://kind.krx.co.kr/external/2002/11/26/K00...,5.등록취소사유\n증권투자회사법 제64조의 규정에 의한 해산사유의 발생\n\n\n,증권투자회사법 제64조의 규정에 의한 해산사유의 발생,0649102002
2,주은리스,030970,2002-11-21 18:24,상장폐지승인,\n\n\n\n\n .document\t{FONT-FAMILY:바탕; FONT-S...,https://kind.krx.co.kr/external/2002/11/21/K00...,5.등록취소사유\n액면가액일정비율 미달로 연속하여 10일간 지속\n\n\n,액면가액일정비율 미달로 연속하여 10일간 지속,0309702002
3,정산애강,022220,2002-11-13 16:59,상장폐지,\n\n\n\n\n .document\t{FONT-FAMILY:바탕; FONT-S...,https://kind.krx.co.kr/external/2002/11/13/K00...,5.등록취소사유\n최종부도 및 은행거래정지\n\n\n,최종부도 및 은행거래정지,0222202002
4,소프트윈,049790,2002-11-06 18:09,상장폐지((주)소프트윈),\n\n\n\n\n .document\t{FONT-FAMILY:바탕; FONT-S...,https://kind.krx.co.kr/external/2002/11/06/K00...,5.등록취소사유\n최종부도 및 은행거래정지\n\n\n,최종부도 및 은행거래정지,0497902002
...,...,...,...,...,...,...,...,...,...
933,유안타제2호스팩,219960,2018-02-08 17:51,상장폐지,\n\n\n\n\n\n\r\n\t.PGBRK\t\t\t{page-break-afte...,https://kind.krx.co.kr/external/2018/02/08/001...,5.상장폐지사유\n상장예비심사 청구서 미제출로 관리종목 지정 후 1개월 이내 동 사...,상장예비심사 청구서 미제출로 관리종목 지정 후 1개월 이내 동 사유 미해소,2199602018
934,셀트리온,068270,2018-02-07 15:48,상장폐지(유가증권시장 상장),\n\n\n\n\n\n\r\n\t.PGBRK\t\t\t{page-break-afte...,https://kind.krx.co.kr/external/2018/02/07/000...,5.상장폐지사유\n유가증권시장 상장\n\n\n,유가증권시장 상장,0682702018
935,키움스팩3호,218710,2018-02-01 18:08,상장폐지,\n\n\n\n\n\n\r\n\t.PGBRK\t\t\t{page-break-afte...,https://kind.krx.co.kr/external/2018/02/01/000...,5.상장폐지사유\n상장예비심사 청구서 미제출로 관리종목 지정 후 1개월 이내 동 사...,상장예비심사 청구서 미제출로 관리종목 지정 후 1개월 이내 동 사유 미해소,2187102018
936,골든브릿지제3호스팩,219580,2018-01-25 18:10,상장폐지,\n\n\n\n\n\n\r\n\t.PGBRK\t\t\t{page-break-afte...,https://kind.krx.co.kr/external/2018/01/25/000...,5.상장폐지사유\n상장예비심사 청구서 미제출로 관리종목 지정 후 1개월 이내 동 사...,상장예비심사 청구서 미제출로 관리종목 지정 후 1개월 이내 동 사유 미해소,2195802018


In [185]:
train['key'] = train['key'].astype(str)

In [186]:
train = train.merge(dataset[['key','text','폐지사유','폐지사유요약']], on='key', how= 'outer')

In [187]:
train['text'] = train['text'].fillna(0)
train['폐지사유'] = train['폐지사유'].fillna(0)
train['폐지사유요약'] = train['폐지사유요약'].fillna(0)
train.shape

(28270, 274)

In [188]:
train = train.dropna()
train.shape

(27377, 274)

In [189]:
train.to_csv(r'C:\Users\JH\Desktop\Final\CBA\Data\kind_bankrupt_Final_ver2.csv', index = False)

# **부도 사유 상세 정리**

In [190]:
dataset = pd.read_csv(r'C:\Users\JH\Desktop\Final\CBA\Data\kind_bankrupt_Final_ver2.csv')

C:\Users\JH\AppData\Roaming\Python\Python39\site-packages\IPython\core\interactiveshell.py:3169: DtypeWarning: Columns (271,272,273) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [191]:
dataset['폐지사유'].value_counts()

0                                                                              24518
0                                                                               2801
2. 정정관련 공시서류 제출일 : \n2020-03-30\n\n\n                                              8
5.상장폐지사유\n기업의 계속성 및 경영의 투명성 등을 종합적으로 고려하여 상장폐지기준에 해당한다고 결정\n\n\n                   6
5.상장폐지사유\n감사의견거절(감사범위제한)\n\n\n                                                     4
5.상장폐지사유\n감사의견 거절(감사범위 제한)\n\n\n                                                   4
5.근거규정\n코스닥시장상장규정 제44조의2\n\n\n                                                     3
5.상장폐지사유\n감사의견거절(감사범위 제한)\n\n\n                                                    3
5.상장폐지사유\n자본전액잠식 등\n\n\n                                                           2
2. 정정관련 공시서류 제출일 : \n2019-08-28\n\n\n                                              2
5.상장폐지사유\n감사의견거절(감사범위 제한 및 계속기업 존속능력에 대한 불확실성)\n\n\n                               2
2. 정정관련 공시서류 제출일 : \n2018-09-27\n\n\n                            